# 7_SCIKIT-LEARN

## 7.1. Основы работы с классами, строящими модели предварительной подготовки данных и модели машинного обучения

Scikit-learn использует объектно-ориентированный подход, где каждая модель — это класс с определенными методами. Давайте разберемся, как это работает.

**Классификаторы vs Регрессоры**  
- Классификаторы: `[Метод]Classifier` (например, `RandomForestClassifier`)
- Регрессоры: `[Метод]Regressor` (например, `LinearRegression`)
- Универсальные: `[Метод]` (например, `KMeans` для кластеризации)

Ключевые принципы
1. Единообразие интерфейса: Все модели имеют методы .fit(), .predict()
2. Независимость от типа данных: Работают с NumPy arrays и pandas DataFrames
3. Совместимость: Можно комбинировать в конвейерах (pipelines)
4. Наследование: Используйте BaseEstimator и TransformerMixin для совместимости

Важные методы
|Метод|Назначение|Возвращает|
|-|-|-|
|`.fit()`|Обучение модели|`self`|
|`.predict()`|Предсказания|Массив предсказаний|
|`.transform()`|Преобразование данных|Преобразованные данные|
|`.fit_transform()`|Обучение + преобразование|Преобразованные данные|
|`.score()`|Оценка качества|Значение метрики|

**Стандартный workflow в scikit-learn**

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# 1. Создание экземпляра класса
model = RandomForestClassifier()
scaler = StandardScaler()

# 2. Обучение модели
scaler.fit(X_train)          # Только признаки
model.fit(X_train, y_train)  # Признаки + метки

# 3. Применение модели
X_scaled = scaler.transform(X_test)
predictions = model.predict(X_scaled)
```

**Создание собственного класса преобразователя**

```python
# Базовый шаблон
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, param1=default_value):
        self.param1 = param1
    
    def fit(self, X, y=None):
        # Вычисление параметров
        return self
    
    def transform(self, X):
        # Применение преобразования
        return X
```

**Пример: Замена пропусков средним (MeanImputer)**

In [1]:
import pandas as pd
import numpy as np

class MeanImputer:
    def __init__(self, copy=True):
        self.copy = copy
        self._encoder_dict = {}
    
    def _is_numpy(self, X):
        return isinstance(X, np.ndarray)
    
    def fit(self, X, y=None):
        # Создаем словарь для хранения средних значений
        self._encoder_dict = {}
        
        # Проверяем тип данных
        is_np = self._is_numpy(X)
        
        # Обработка 1D массива
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        
        # Вычисляем средние значения
        if is_np:
            for col in range(X.shape[1]):
                self._encoder_dict[col] = np.nanmean(X[:, col])
        else:
            for col in X.columns:
                self._encoder_dict[col] = X[col].mean()
        
        return self
    
    def transform(self, X):
        if self.copy:
            X = X.copy()
        
        is_np = self._is_numpy(X)
        
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        
        # Замена пропусков
        if is_np:
            for col in range(X.shape[1]):
                X[:, col] = np.nan_to_num(X[:, col], nan=self._encoder_dict[col])
        else:
            for col in X.columns:
                X[col] = np.where(X[col].isnull(), self._encoder_dict[col], X[col])
        
        return X

In [2]:
# Использование MeanImputer

# Пример с DataFrame
toy_train = pd.DataFrame({
    'Balance': [8.3, np.nan, 10.2, 3.1],
    'Age': [23, 29, 36, np.nan]
})

# Создание и применение преобразователя
imp = MeanImputer()
imp.fit(toy_train)
toy_train_inputed = imp.transform(toy_train)
print(toy_train_inputed)

   Balance        Age
0      8.3  23.000000
1      7.2  29.000000
2     10.2  36.000000
3      3.1  29.333333


**Создание собственной модели машинного обучения**

Пример: K-Nearest Neighbors (KNN)

In [3]:
import math

class KNN_Estimator:
    def __init__(self, k=5, task='classification'):
        self.k = k
        self.task = task
    
    def _euclidean_distance(self, x1, x2):
        """Вычисляет евклидово расстояние между двумя точками"""
        distance = 0
        for i in range(len(x1)):
            distance += (x1[i] - x2[i]) ** 2
        return math.sqrt(distance)
    
    def _vote(self, neighbor_labels):
        """Голосование для классификации"""
        counts = np.bincount(neighbor_labels.astype('int'))
        return counts.argmax()
    
    def fit(self, X, y):
        """Запоминаем обучающие данные"""
        self.X_train = X
        self.y_train = y
        return self
    
    def predict(self, X):
        """Предсказание для новых данных"""
        predictions = []
        
        for test_sample in X:
            # Находим расстояния до всех точек
            distances = [self._euclidean_distance(test_sample, x) 
                        for x in self.X_train]
            
            # Находим k ближайших соседей
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = self.y_train[nearest_indices]
            
            if self.task == 'classification':
                # Голосование для классификации
                pred = self._vote(nearest_labels)
            else:
                # Среднее для регрессии
                pred = np.mean(nearest_labels)
            
            predictions.append(pred)
        
        return np.array(predictions)

In [4]:
# Использование KNN модели

# Данные для классификации
X_train = np.array([[1, 2], [2, 3], [3, 1], [4, 2]])
y_train = np.array([0, 0, 1, 1])

X_test = np.array([[2.5, 2]])

# Обучение и предсказание
knn = KNN_Estimator(k=3, task='classification')
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
print(f"Предсказание: {predictions}")

Предсказание: [0]


## 7.2. Строим свой первый конвейер моделей

**Конвейер (pipeline)** — это способ объединить несколько шагов обработки данных и моделирования в одну последовательность. Представьте, что это конвейер на заводе, где данные проходят несколько станций обработки перед тем как стать готовым продуктом (моделью).

In [5]:
# импортируем библиотеки pandas, numpy
import pandas as pd
import numpy as np
# импортируем модуль os и функцию train_test_split()
import os
from sklearn.model_selection import train_test_split
# импортируем класс StandardScaler,
# выполняющий стандартизацию
from sklearn.preprocessing import StandardScaler
# импортируем класс LogisticRegression
from sklearn.linear_model import LogisticRegression

```python
# взглянем на наш рабочий каталог
os.getcwd()
```

**Ключевые правила**

- Сначала разделяй, потом обрабатывай — избегай утечки данных
- Обучай преобразования только на обучающих данных — тестовые данные должны быть "невидимы"
- Используй конвейеры — для удобства и предотвращения ошибок
- Настраивай гиперпараметры на валидационной выборке — тестовая только для финальной оценки
- Интерпретируй коэффициенты после стандартизации — они показывают влияние в стандартных отклонениях

Конвейер — это как рецепт: все ингредиенты обрабатываются в правильной последовательности, чтобы получить вкусный результат! 🚀🍳

**Зачем нужен конвейер?**

Без конвейера:
```python
# 1. Стандартизируем данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Обучаем модель
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# 3. Делаем прогноз
predictions = model.predict(X_test_scaled)
```
С конвейером:

```python
# Все в одной строке!
pipeline = make_pipeline(StandardScaler(), LogisticRegression())
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
```

**Основные проблемы без конвейера**

Утечка данных (data leakage)
Если сначала обработать ВСЕ данные, а потом разделить на обучающую/тестовую выборки, модель "узнает" о тестовых данных заранее:

```python
# ❌ НЕПРАВИЛЬНО: утечка данных
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)  # Обрабатываем все данные
X_train, X_test = train_test_split(X_all_scaled)  # Потом делим

# ✅ ПРАВИЛЬНО: сначала делим, потом обрабатываем
X_train, X_test = train_test_split(X_all)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Только на обучающих!
X_test_scaled = scaler.transform(X_test)        # Применяем параметры с обучения
```

**Разбиение данных на выборки**

Три типа выборок:
- Обучающая — для обучения модели
- Валидационная — для настройки гиперпараметров
- Тестовая — для финальной оценки

```python
from sklearn.model_selection import train_test_split

# Разделяем данные
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1),  # Признаки (без целевой переменной)
    data['Response'],               # Целевая переменная
    test_size=0.3,                  # 30% в тестовую выборку
    stratify=data['Response'],      # Сохраняем пропорции классов
    random_state=42                 # Для воспроизводимости
)
```

**Параметры vs Гиперпараметры**

Параметры — то, что модель "учит" сама:  
📊 Средние значения для заполнения пропусков  
📈 Регрессионные коэффициенты в логистической регрессии  
🌳 Правила разбиения в дереве решений

Гиперпараметры — то, что задаем МЫ:  
🔢 Количество соседей в KNN (k=5)  
💪 Сила регуляризации в логистической регрессии (C=10)  
🎯 Максимальная глубина дерева (max_depth=5)

**Стандартизация данных**

Зачем нужна? Чтобы признаки были в одном масштабе:

```python
from sklearn.preprocessing import StandardScaler

# Создаем и обучаем scaler ТОЛЬКО на обучающих данных
scaler = StandardScaler()
scaler.fit(X_train)  # Вычисляем средние и стандартные отклонения

# Применяем к обеим выборкам
X_train_scaled = scaler.transform(X_train)  # Обучающая
X_test_scaled = scaler.transform(X_test)    # Тестовая (те же параметры!)
```

Формула стандартизации:

```text
x_стандартизированный = (x - среднее) / стандартное_отклонение
```

**Обучение модели**
```python
from sklearn.linear_model import LogisticRegression

# Создаем и обучаем модель
model = LogisticRegression(solver='lbfgs', max_iter=200)
model.fit(X_train_scaled, y_train)

# Оцениваем качество
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_test)

print(f"Качество на обучении: {train_score:.3f}")
print(f"Качество на тесте: {test_score:.3f}")
```

**Интерпретация коэффициентов**

После стандартизации коэффициенты показывают влияние на логит при изменении признака на одно стандартное отклонение:

```python
# Коэффициенты модели
coefficients = model.coef_[0]
feature_names = X_train.columns

for coef, feature in zip(coefficients, feature_names):
    print(f"{feature}: {coef:.3f}")
```
Пример интерпретации:
- Number of Policies: -0.109 — увеличение количества полисов на 1 стандартное отклонение УМЕНЬШАЕТ шансы отклика
- Income: 0.042 — увеличение дохода на 1 стандартное отклонение УВЕЛИЧИВАЕТ шансы отклика

**Полный рабочий процесс**

In [6]:
# 1. Загрузка данных
data = pd.read_csv('data/StateFarm.csv', sep=';')

In [7]:
# 2. Разделение на признаки и целевую переменную
X = data.drop('Response', axis=1)
y = data['Response']

In [8]:
# 3. Разделение на обучающую/тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# 4. Стандартизация
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# 5. Обучение модели
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [11]:
# 6. Оценка качества
print(f"Качество на тесте: {model.score(X_test_scaled, y_test):.3f}")

Качество на тесте: 0.900
